# Reference

Before we discuss what `reference` means, lets start with a small exercise. We have a list `lst1`

In [22]:
lst1 = [1, 32, 3, 43]

Now, lets create `lst2` using assination to `lst1`

In [23]:
lst2 = lst1
print(lst1, lst2)

[1, 32, 3, 43] [1, 32, 3, 43]


What we have now is that `lst1` and `lst2` pointing to same memory location, its not a copy but two identifiers pointing to a single memory location. We can prove it by using `id` function 

In [13]:
print(id(lst1), id(lst2))

140557574935944 140557574935944


Now, lets try to change the value of one element of `lst1` and see its effect of `lst2`

In [14]:
lst2[3] = "TEST"
print(lst1, lst2)
print(id(lst1), id(lst2))

[1, 32, 3, 'TEST'] [1, 32, 3, 'TEST']
140557574935944 140557574935944


What we observed is that, since both the identifier's were pointing to same memory location, change in one gets refected in another. 

Now what happens if we delete one of the identifier. Lets try

In [15]:
del(lst1)

So we deleted identifier `lst1` and now lets check if `lst2` still exists

In [16]:
print(lst2, id(lst2))

[1, 32, 3, 'TEST'] 140557574935944


so, `lst2` and its value still lives 

In [24]:
del(lst2)

So, now we have deleted `lst2` also, trying to access it will result in error as shown below

```python
print(id(lst2))
```

```python
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<ipython-input-7-b9dd8f0bc209> in <module>()
----> 1 print(id(lst2))

NameError: name 'lst2' is not defined

```

Lets try to find the reasons behind the above beharior. 

### What is Strong reference

When memory is referenced by a variable, its called **strong reference**. What that means is that untill all the strong references are removed from the memory location, it is not removed.

In [26]:
a = 101
b = a
print(a, id(a))
print(b, id(b))

101 140557821047136
101 140557821047136


In [27]:
del(a)
# print(a, id(a))
print(b, id(b))

101 140557821047136


In [29]:
del(b)

NameError: name 'b' is not defined

In [34]:
import sys

class MayaHello:
    def __init__(self, lang='hi'):
        self.lang = lang
    
    def hello(self):
        if(self.lang == 'hi'):
            return "नमस्ते"
        elif(self.lang == 'de'):
            return "Hallo"
        else:
            return ("Hello")
    
    def __del__(self):
        print("self destruct initiated")
        del self.lang

a = MayaHello('hi')
print(a.hello())
sys.stdout.flush()
b = a 
print(b.hello()) 
sys.stdout.flush()

नमस्ते
self destruct initiated
नमस्ते


In [35]:
print(a.hello())

नमस्ते


## Weak Reference

We have seen that strong reference, increases the reference count, where as Weak references have no effect on the reference count for an object. 

Weak reference is Non-permanent References to Objects and is accomplised with the help of `weakref` module in the Python standard library.

Python garbage collector can free the memory if it's only referenced by weak references, However, till that happens weak reference may return the object, but its not guaranteed.

Lets try the examples, which we created in strong reference and see that happens to them when we use `weakref` instead.

In [53]:
import weakref

a = MayaHello()

b = weakref.ref(a)
print(a, id(a))
print(b, id(b))

self destruct initiated
<__main__.MayaHello object at 0x7fd61c556550> 140557575087440
<weakref at 0x7fd61c4d1048; to 'MayaHello' at 0x7fd61c556550> 140557574541384


In [54]:
print(a.hello())

नमस्ते


Now, lets delete the `b` identifier

In [55]:
del(b)

and, check its effect if any on `a`

In [57]:
print(a.hello())

नमस्ते


Lets create another weakref of a and when try to delete `a` and check its effect in weakref created.

In [58]:
b = weakref.ref(a)
print(a, id(a))
print(b, id(b))

<__main__.MayaHello object at 0x7fd61c556550> 140557575087440
<weakref at 0x7fd61c4d1638; to 'MayaHello' at 0x7fd61c556550> 140557574542904


In [59]:
del(a)

self destruct initiated


```python
print(b.hello())
```

```
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
<ipython-input-60-38ed82fe8e5b> in <module>()
----> 1 print(b.hello())

AttributeError: 'weakref' object has no attribute 'hello'

```

### When to Use 

Weak references are used 
- to refer to objects which are expensive in nature and thus copying is not advised
- to implement caches

Weak references to objects are managed through the ref class. To retrieve the original object, call the reference object.

** Reference URL's **
- https://stackoverflow.com/questions/2436302/when-to-use-weak-references-in-python
- https://stackoverflow.com/questions/1507566/how-and-when-to-appropriately-use-weakref-in-python


The typical use for weak references is if A has a reference to B and B has a reference to A. Without a proper cycle-detecting garbage collector, those two objects would never get GC'd even if there are no references to either from the "outside". However if one of the references is "weak", the objects will get properly GC'd.

However, Python does have a cycle-detecting garbage collector (since 2.0!), so that doesn't count :)

Another use for weak references is for caches. It's mentioned in the weakref documentation:

A primary use for weak references is to implement caches or mappings holding large objects, where it’s desired that a large object not be kept alive solely because it appears in a cache or mapping.
If the GC decides to destroy one of those objects, and you need it, you can just recalculate / refetch the data.

As a more transparent alternative to weakref.ref, we can use weakref.proxy. This call requires a strong reference to an object as its first argument and returns a weak reference proxy. The proxy behaves just like a strong reference, but throws an exception when used after the target is dead:

In [53]:
obj = ExpensiveObject()
b = weakref.proxy(obj)
del(obj)
# print('obj:', obj)
print('ref proxy:', b) # Pointing to None Type

(Deleting <__main__.ExpensiveObject instance at 0x7f40e6ffa1b8>)
('ref proxy:', <weakproxy at 0x7f40e6f79310 to NoneType at 0x7f41010c3e40>)


### Reference Callbacks

The ref constructor accepts an optional callback function that is invoked when the referenced object is deleted.



In [63]:
import weakref


class ExpensiveObject:

    def __del__(self):
        print('(Deleting {})'.format(self))


def callback(reference):
    """Invoked when referenced object is deleted"""
    print('callback({!r})'.format(reference))


obj = ExpensiveObject()
r = weakref.ref(obj, callback)

print('obj:', obj)
print('ref:', r)
print('r():', r())

print('deleting obj')
del obj
print('r():', r())

obj: <__main__.ExpensiveObject object at 0x7fd61c4dd198>
ref: <weakref at 0x7fd61c4de228; to 'ExpensiveObject' at 0x7fd61c4dd198>
r(): <__main__.ExpensiveObject object at 0x7fd61c4dd198>
deleting obj
(Deleting <__main__.ExpensiveObject object at 0x7fd61c4dd198>)
callback(<weakref at 0x7fd61c4de228; dead>)
r(): None


## References

- https://mindtrove.info/python-weak-references/